# 管理数据库

>与传统的数据库引擎类似，您也可以在Milvus中创建数据库，并将权限分配给某些用户来管理它们。然后，这些用户有权管理数据库中的集合。Milvus集群最多支持64个数据库。

## 创建数据库

要创建数据库，您需要首先连接到Milvus集群并为其准备一个名称：

In [ ]:
from pymilvus import connections, db

conn = connections.connect(host="127.0.0.1", port=19530)

database = db.create_database("book")

## 使用数据库

Milvus集群附带一个名为"default"的默认数据库。除非另外指定，否则集合将在默认数据库中创建。

要更改默认数据库，请执行以下操作：

In [ ]:
db.using_database("book")

您还可以设置连接到Milvus集群时使用的数据库，如下所示：

In [ ]:
conn = connections.connect(
    host="127.0.0.1",
    port="19530",
    db_name="default"
)

## 列出数据库

要查找Milvus集群中的所有现有数据库，请执行以下操作：

In [ ]:
db.list_database()

## Drop database

要删除一个数据库，必须先删除它的所有集合。否则，丢弃失败。

In [ ]:
db.drop_database("book")

In [ ]:
db.list_database()

## 将RBAC用于数据库

ilvus 是一个开源的向量数据库，用于存储和检索大规模高维向量数据。RBAC 在 Milvus 中的作用是限制用户对 Milvus 数据库和操作的访问权限，以确保数据的安全性和保密性。

通过 RBAC，可以定义不同的角色和权限，将用户分配到不同的角色中，从而控制他们对 Milvus 中的资源和操作的访问。例如，可以创建一个管理员角色，拥有对所有资源和操作的完全权限；同时，可以创建一个只读用户角色，仅允许对数据进行查询而不允许进行写入操作。

RBAC 在 Milvus 中可以通过配置和管理用户、角色和权限来实现。管理员可以创建、编辑和删除角色，分配角色给用户，并为每个角色分配相应的权限。这样，可以根据具体的需求和安全策略，灵活地控制用户对 Milvus 数据库的访问权限。

RBAC还涵盖数据库操作并确保向前兼容性。Permission API（授权/撤销/列表授权）中的单词数据库具有以下含义：

- 如果Milvus连接和Permission API调用都没有指定db_name，则数据库引用默认数据库。
- 如果一个Milvus连接指定了一个db_name，但之后的Permission API调用没有指定，那么database引用的数据库的名称是在Milvus连接中指定的。
- 如果在Milvus连接上进行Permission API调用，无论是否指定了db_name，数据库都引用其名称在Permission API调用中指定的数据库。


下面的代码片段在下面列出的块中共享。

In [ ]:
from pymilvus import connections, Role

_HOST = '127.0.0.1'
_PORT = '19530'
_ROOT = "root"
_ROOT_PASSWORD = "Milvus"
_ROLE_NAME = "test_role"
_PRIVILEGE_INSERT = "Insert"

def connect_to_milvus(db_name="default"):
    print(f"connect to milvus\n")
    connections.connect(host=_HOST, port=_PORT, user=_ROOT, password=_ROOT_PASSWORD, db_name=db_name)

如果Milvus连接和Permission API调用都没有指定db_name，则数据库引用默认数据库。

In [ ]:
connect_to_milvus()
role = Role(_ROLE_NAME)
# role.create()

In [ ]:
connect_to_milvus()
role.grant("Collection", "*", _PRIVILEGE_INSERT)

print(role.list_grants())
print(role.list_grant("Collection", "*"))

role.revoke("Collection", "*", privilege=_PRIVILEGE_INSERT)

print(role.list_grants())
print(role.list_grant("Collection", "*"))

如果一个Milvus连接指定了一个db_name，但之后的Permission API调用没有指定，那么database引用的数据库的名称是在Milvus连接中指定的。


In [ ]:
from pymilvus import connections, db

conn = connections.connect(host="127.0.0.1", port=19530)
database = db.create_database("foo")

In [ ]:
# NOTE: please make sure the 'foo' db has been created
connect_to_milvus(db_name="foo")
# This role will have the insert permission of all collections under foo db,
# excluding the insert permissions of collections under other dbs
role.grant("Collection", "*", _PRIVILEGE_INSERT)
print(role.list_grants())
print(role.list_grant("Collection", "*"))
role.revoke("Collection", "*", _PRIVILEGE_INSERT)
print(role.list_grants())
print(role.list_grant("Collection", "*"))

如果在Milvus连接上进行Permission API调用，无论是否指定了db_name，数据库都引用其名称在Permission API调用中指定的数据库。

In [ ]:
# NOTE: please make sure the 'foo' db has been created
db_name = "foo"
connect_to_milvus()
role.grant("Collection", "*", _PRIVILEGE_INSERT, db_name=db_name)
print(role.list_grants(db_name=db_name))
print(role.list_grant("Collection", "*", db_name=db_name))
role.revoke("Collection", "*", _PRIVILEGE_INSERT, db_name=db_name)

In [ ]:
db.drop_database("foo")

----